In [1]:
%cd ..

/home/anton/pancluster


In [112]:
import functools as ft

import numpy as np
import pandas as pd
from sklearn import (
    ensemble,
    feature_selection,
    model_selection,
    pipeline,
    tree
)

## Dataset loading

In [6]:
target = pd.read_csv(
    "accessions.tsv",
    sep="\t",
    index_col="genome"
)["fsp"]
target

genome
GCA_000149955.2     lycopersici
GCA_000222805.1    conglutinans
GCA_000259975.2     lycopersici
GCA_000260175.2     vasinfectum
GCA_000260215.2    conglutinans
                       ...     
GCA_032878545.1     vasinfectum
GCA_032991405.1     vasinfectum
GCA_034509825.1         cubense
GCA_036785135.1    conglutinans
GCA_038050555.1     vasinfectum
Name: fsp, Length: 242, dtype: object

In [ ]:
# pangenome = pd.read_csv(
#     "data/pangenome/pangenome.full.tsv",
#     sep="\t",
#     index_col="genome"
# )
# pangenome

In [5]:
raw_pancluster = pd.read_csv(
    "data/pancluster/pancluster.raw.tsv",
    sep="\t",
    index_col="genome"
)
raw_pancluster

,g0,g22,g23,g24,g26,g27,g28,g29,g30,g32,...,g2263,g2264,g2266,g2267,g2269,g2270,g2271,g2272,g2273,g2275
genome,,,,,,,,,,,,,,,,,,,,,
GCA_000149955.2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_000222805.1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_000259975.2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_000260175.2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_000260215.2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCA_032878545.1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_032991405.1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_034509825.1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
grouped_pancluster = pd.read_csv(
    "data/pancluster/pancluster.grouped.tsv",
    sep="\t",
    index_col="genome"
)
grouped_pancluster

,c10021-g1069,c10021-g1201,c10021-g1210,c10021-g1362,c10021-g1696,c10021-g2046,c10021-g234,c10021-g263,c10021-g265,c10021-g289,...,c9703-g504,c9703-g513,c9703-g524,c9724-g109,c9724-g394,c9727-g858,c9727-g868,c9727-g874,c9727-g886,c9727-g900
genome,,,,,,,,,,,,,,,,,,,,,
GCA_000149955.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_000222805.1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
GCA_000259975.2,0,0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
GCA_000260175.2,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
GCA_000260215.2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCA_032878545.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_032991405.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_034509825.1,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
ordered_pancluster = pd.read_csv(
    "data/pancluster/pancluster.ordered.tsv",
    sep="\t",
    index_col="genome"
)
ordered_pancluster

,c10021-g1069,c10021-g1201,c10021-g1201 c10021-g1362,c10021-g1210,c10021-g1362,c10021-g2046,c10021-g234,c10021-g234 c10021-g273,c10021-g234 c10021-g273 c10021-g325,c10021-g234 c10021-g273 c10021-g898,...,c9703-g524,c9724-g109,c9724-g370 c9724-g374 c9724-g394,c9727-g858,c9727-g859 c9727-g864 c9727-g874,c9727-g864 c9727-g874 c9727-g868,c9727-g868,c9727-g868 c9727-g900,c9727-g874 c9727-g868 c9727-g900,c9727-g886
genome,,,,,,,,,,,,,,,,,,,,,
GCA_000149955.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_000222805.1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
GCA_000259975.2,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_000260175.2,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
GCA_000260215.2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCA_032878545.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_032991405.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GCA_034509825.1,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
dataframes = [raw_pancluster, grouped_pancluster, ordered_pancluster]

## Grid definitions

In [76]:
def logstep(start: int, end: int, step: float):
    steps = []
    current = start
    power = 1
    while current < end:
        steps.append(current)
        power += step
        current = round(start**power)
    steps.append(end)
    return steps

In [12]:
cv = model_selection.StratifiedKFold(4, shuffle=True, random_state=0)
cv

StratifiedKFold(n_splits=4, random_state=0, shuffle=True)

In [95]:
score_func = ft.partial(
    feature_selection.mutual_info_classif,
    discrete_features=True
)
score_func

functools.partial(<function mutual_info_classif at 0x7f787e5bc900>, discrete_features=True)

In [136]:
tree_grid = {
    "criterion": ["gini", "entropy", "log_loss"],
    "splitter": ["best", "random"],
    "max_depth": [1, 3, 5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5, 10],
    "max_features": ["sqrt", "log2", None],
    "class_weight": ["balanced", None],
    "random_state": [0]
}
adaboost_grid = {
    "estimator": [tree.DecisionTreeClassifier()],
    "learning_rate": [0.01, 0.1, 1.0],
    "n_estimators": [50, 100, 200],
    "random_state": [0],
    **{f"estimator__{key}": value for key, value in tree_grid.items()}
}
forest_grid = {
    "n_estimators": [50, 100, 200],
    **{key: value for key, value in tree_grid.items() if key != "splitter"}
}
grids = [tree_grid, adaboost_grid, forest_grid]
grids

[{'criterion': ['gini', 'entropy', 'log_loss'],
  'splitter': ['best', 'random'],
  'max_depth': [1, 3, 5, 10, 20, None],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 5, 10],
  'max_features': ['sqrt', 'log2', None],
  'class_weight': ['balanced', None],
  'random_state': [0]},
 {'estimator': [DecisionTreeClassifier()],
  'learning_rate': [0.01, 0.1, 1.0],
  'n_estimators': [50, 100, 200],
  'random_state': [0],
  'estimator__criterion': ['gini', 'entropy', 'log_loss'],
  'estimator__splitter': ['best', 'random'],
  'estimator__max_depth': [1, 3, 5, 10, 20, None],
  'estimator__min_samples_split': [2, 5, 10],
  'estimator__min_samples_leaf': [1, 2, 5, 10],
  'estimator__max_features': ['sqrt', 'log2', None],
  'estimator__class_weight': ['balanced', None],
  'estimator__random_state': [0]},
 {'n_estimators': [50, 100, 200],
  'criterion': ['gini', 'entropy', 'log_loss'],
  'max_depth': [1, 3, 5, 10, 20, None],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': 

In [134]:
tree_pipeline = pipeline.Pipeline([
    ("select", feature_selection.SelectKBest()),
    ("model", tree.DecisionTreeClassifier())
])
adaboost_pipeline = pipeline.Pipeline([
    ("select", feature_selection.SelectKBest()),
    ("model", ensemble.AdaBoostClassifier())
])
forest_pipeline = pipeline.Pipeline([
    ("select", feature_selection.SelectKBest()),
    ("model", ensemble.RandomForestClassifier())
])
pipelines = [tree_pipeline, adaboost_pipeline, forest_pipeline]
pipelines

[Pipeline(steps=[('select', SelectKBest()), ('model', DecisionTreeClassifier())]),
 Pipeline(steps=[('select', SelectKBest()), ('model', AdaBoostClassifier())]),
 Pipeline(steps=[('select', SelectKBest()), ('model', RandomForestClassifier())])]

In [ ]:
def create_search(
    dataframe: pd.DataFrame,
    target: pd.Series,
    pipeline: pipeline.Pipeline,
    grid: dict
):
    grid = {
        "select__score_func": [score_func],
        "select__k": logstep(10, dataframe.shape[1], 0.25),
        **{f"model__{key}": value for key, value in grid.items()}
    }
    return model_selection.GridSearchCV(
        estimator=pipeline,
        param_grid=grid,
        scoring="f1_weighted",
        n_jobs=10,
        verbose=1
    ).fit(dataframe, target).cv_results_

In [135]:
grid_searches = [
    create_search(dataframe, target, estimator, grid)
    for estimator, grid in zip([pipelines[1]], [grids[1]])
    for dataframe in dataframes
]
grid_searches

Fitting 5 folds for each of 77760 candidates, totalling 388800 fits


KeyboardInterrupt: 